In [9]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from urllib.parse import urljoin
import json
import re
import geopandas as gpd
from shapely.geometry import Point

In [10]:
# urls of metro manila cities
# cities with >1500 houses are separated into barangays

urls = ["/metro-manila/caloocan/house/buy/",
"/metro-manila/valenzuela/house/buy/",
"/metro-manila/malabon/house/buy/",
"/metro-manila/marikina/house/buy/",
"/metro-manila/quezon-city/fairview/house/buy/",
"/metro-manila/quezon-city/cubao-1/house/buy/",
"/metro-manila/quezon-city/tandang-sora-3/house/buy/",
"/metro-manila/quezon-city/batasan-hills/house/buy/",
"/metro-manila/quezon-city/commonwealth/house/buy/",
"/metro-manila/quezon-city/holy-spirit/house/buy/",
"/metro-manila/quezon-city/project-8/house/buy/",
"/metro-manila/quezon-city/diliman/house/buy/",
"/metro-manila/quezon-city/teachers-village-east/house/buy/",
"/metro-manila/navotas/house/buy/",
"/metro-manila/manila/house/buy/",
"/metro-manila/san-juan-5/house/buy/",
"/metro-manila/mandaluyong/house/buy/",
"/metro-manila/pasig/san-antonio-106/house/buy/",
"/metro-manila/pasig/ortigas-cbd-1/house/buy/",
"/metro-manila/pasig/san-miguel-80/house/buy/",
"/metro-manila/pasig/ugong/house/buy/",
"/metro-manila/pasig/pinagbuhatan/house/buy/",
"/metro-manila/pasig/rosario-14/house/buy/",
"/metro-manila/pasig/kapitolyo/house/buy/",
"/metro-manila/pasig/bagong-ilog/house/buy/",
"/metro-manila/pasig/santolan/house/buy/",
"/metro-manila/pasig/ortigas-avenue/house/buy/",
"/metro-manila/makati/rockwell-1/house/buy/",
"/metro-manila/makati/san-lorenzo-16/house/buy/",
"/metro-manila/makati/san-antonio-104/house/buy/",
"/metro-manila/makati/poblacion-360/house/buy/",
"/metro-manila/makati/salcedo-village-1/house/buy/",
"/metro-manila/makati/bel-air/house/buy/",
"/metro-manila/makati/legazpi-village-1/house/buy/",
"/metro-manila/makati/dasmarinas-2/house/buy/",
"/metro-manila/makati/urdaneta-2/house/buy/",
"/metro-manila/pasay/house/buy/",
"/metro-manila/taguig/house/buy/",
"/metro-manila/paranaque/b-f-homes/house/buy/",
"/metro-manila/paranaque/don-bosco/house/buy/",
"/metro-manila/paranaque/moonwalk/house/buy/",
"/metro-manila/paranaque/merville/house/buy/",
"/metro-manila/paranaque/tambo-13/house/buy/",
"/metro-manila/paranaque/san-antonio-105/house/buy/",
"/metro-manila/paranaque/san-isidro-184/house/buy/",
"/metro-manila/paranaque/sun-valley/house/buy/",
"/metro-manila/paranaque/marcelo-green-village/house/buy/",
"/metro-manila/paranaque/san-dionisio-4/house/buy/",
"/metro-manila/las-pinas/talon-dos/house/buy/",
"/metro-manila/las-pinas/almanza-uno/house/buy/",
"/metro-manila/las-pinas/almanza-dos/house/buy/",
"/metro-manila/las-pinas/pilar-12/house/buy/",
"/metro-manila/las-pinas/pulang-lupa-dos/house/buy/",
"/metro-manila/las-pinas/pamplona-tres/house/buy/",
"/metro-manila/las-pinas/pamplona-dos/house/buy/",
"/metro-manila/las-pinas/talon-singko/house/buy/", 
"/metro-manila/las-pinas/talon-kuatro/house/buy/",
"/metro-manila/las-pinas/pulang-lupa-uno/house/buy/",
"/metro-manila/muntinlupa/alabang/house/buy/",
"/metro-manila/muntinlupa/sucat/house/buy/",
"/metro-manila/muntinlupa/ayala-alabang/house/buy/",
"/metro-manila/muntinlupa/poblacion-362/house/buy/",
"/metro-manila/muntinlupa/buli-3/house/buy/",
"/metro-manila/muntinlupa/tunasan/house/buy/",
"/metro-manila/muntinlupa/cupang-4/house/buy/",
"/metro-manila/muntinlupa/putatan/house/buy/",
"/metro-manila/pateros/house/buy/"]

In [11]:
# initialization of columns

cities = []
titles = []
prices = []
latitudes = []
longitudes = []
listing_links = []
addresses = []
floor_areas = []
land_areas = []
number_bathrooms = []
number_bedrooms = []
subcategories = []
agent_names = []
verifications = []
agencies = []

In [12]:
#looping through the urls and extracting relevant housing information

base_url = "https://www.lamudi.com.ph"

for url in urls:
    re_city = re.search(r'\.ph/(.*?)/house', urljoin(base_url, url))
    city = re_city.group(1)
    while url:
        response = requests.get(urljoin(base_url, url))
        soup = BeautifulSoup(response.text, 'html.parser')
        listings = soup.find_all("div", attrs={"class":"ListingCell-AllInfo ListingUnit"})
        for listing in listings:
            coord = listing["data-geo-point"]
            coord = coord.strip("[]")
            lon, lat = map(float, coord.split(","))
            longitudes.append(lon)
            latitudes.append(lat)

            price = listing["data-price"]
            prices.append(float(price))
            
            cities.append(city)
            
            title = listing.find("h3", class_="ListingCell-KeyInfo-title")["title"]
            titles.append(title)

            address = listing.find("span", class_="ListingCell-KeyInfo-address-text").text.strip()
            addresses.append(address)

            listing_link = listing.find("a", class_="js-listing-link")["href"]
            listing_links.append(listing_link)

            floor_area = listing.get("data-building_size", None)
            floor_areas.append(floor_area)

            land_area = listing.get("data-land_size", None)
            land_areas.append(land_area)

            number_bathroom = listing.get("data-bathrooms", None)
            number_bathrooms.append(number_bathroom)

            number_bedroom = listing.get("data-bedrooms", None)
            number_bedrooms.append(number_bedroom)
            
            subcategory = listing["data-subcategories"]
            subcategories.append(subcategory)
                        
            agent_tag = listing.find("div", class_="ListingDetail-agent-name")
            agent_names.append(agent_tag.text.strip())
            
            verification_div_tag = listing.find("div", class_="verification-badge lmd-verification-tooltip")
            verification_span_tag = verification_div_tag.find("span")
            verification_text = verification_span_tag.get_text(strip=True)
            verifications.append(verification_text)          
        
        next_tag = soup.find("link", {"rel":"next"})
        if next_tag is not None:
            url = next_tag["href"]
        else:
            url = None
            
        print(url)
    

https://www.lamudi.com.ph/metro-manila/caloocan/house/buy/?page=2
https://www.lamudi.com.ph/metro-manila/caloocan/house/buy/?page=3
https://www.lamudi.com.ph/metro-manila/caloocan/house/buy/?page=4
https://www.lamudi.com.ph/metro-manila/caloocan/house/buy/?page=5
https://www.lamudi.com.ph/metro-manila/caloocan/house/buy/?page=6
https://www.lamudi.com.ph/metro-manila/caloocan/house/buy/?page=7
https://www.lamudi.com.ph/metro-manila/caloocan/house/buy/?page=8
https://www.lamudi.com.ph/metro-manila/caloocan/house/buy/?page=9
https://www.lamudi.com.ph/metro-manila/caloocan/house/buy/?page=10
https://www.lamudi.com.ph/metro-manila/caloocan/house/buy/?page=11
https://www.lamudi.com.ph/metro-manila/caloocan/house/buy/?page=12
https://www.lamudi.com.ph/metro-manila/caloocan/house/buy/?page=13
https://www.lamudi.com.ph/metro-manila/caloocan/house/buy/?page=14
https://www.lamudi.com.ph/metro-manila/caloocan/house/buy/?page=15
https://www.lamudi.com.ph/metro-manila/caloocan/house/buy/?page=16
Non

https://www.lamudi.com.ph/metro-manila/quezon-city/tandang-sora-3/house/buy/?page=9
https://www.lamudi.com.ph/metro-manila/quezon-city/tandang-sora-3/house/buy/?page=10
https://www.lamudi.com.ph/metro-manila/quezon-city/tandang-sora-3/house/buy/?page=11
https://www.lamudi.com.ph/metro-manila/quezon-city/tandang-sora-3/house/buy/?page=12
https://www.lamudi.com.ph/metro-manila/quezon-city/tandang-sora-3/house/buy/?page=13
https://www.lamudi.com.ph/metro-manila/quezon-city/tandang-sora-3/house/buy/?page=14
https://www.lamudi.com.ph/metro-manila/quezon-city/tandang-sora-3/house/buy/?page=15
https://www.lamudi.com.ph/metro-manila/quezon-city/tandang-sora-3/house/buy/?page=16
https://www.lamudi.com.ph/metro-manila/quezon-city/tandang-sora-3/house/buy/?page=17
https://www.lamudi.com.ph/metro-manila/quezon-city/tandang-sora-3/house/buy/?page=18
https://www.lamudi.com.ph/metro-manila/quezon-city/tandang-sora-3/house/buy/?page=19
https://www.lamudi.com.ph/metro-manila/quezon-city/tandang-sora-3/

https://www.lamudi.com.ph/metro-manila/quezon-city/project-8/house/buy/?page=9
https://www.lamudi.com.ph/metro-manila/quezon-city/project-8/house/buy/?page=10
https://www.lamudi.com.ph/metro-manila/quezon-city/project-8/house/buy/?page=11
https://www.lamudi.com.ph/metro-manila/quezon-city/project-8/house/buy/?page=12
https://www.lamudi.com.ph/metro-manila/quezon-city/project-8/house/buy/?page=13
https://www.lamudi.com.ph/metro-manila/quezon-city/project-8/house/buy/?page=14
https://www.lamudi.com.ph/metro-manila/quezon-city/project-8/house/buy/?page=15
https://www.lamudi.com.ph/metro-manila/quezon-city/project-8/house/buy/?page=16
https://www.lamudi.com.ph/metro-manila/quezon-city/project-8/house/buy/?page=17
https://www.lamudi.com.ph/metro-manila/quezon-city/project-8/house/buy/?page=18
https://www.lamudi.com.ph/metro-manila/quezon-city/project-8/house/buy/?page=19
https://www.lamudi.com.ph/metro-manila/quezon-city/project-8/house/buy/?page=20
https://www.lamudi.com.ph/metro-manila/qu

https://www.lamudi.com.ph/metro-manila/pasig/san-miguel-80/house/buy/?page=8
https://www.lamudi.com.ph/metro-manila/pasig/san-miguel-80/house/buy/?page=9
https://www.lamudi.com.ph/metro-manila/pasig/san-miguel-80/house/buy/?page=10
https://www.lamudi.com.ph/metro-manila/pasig/san-miguel-80/house/buy/?page=11
https://www.lamudi.com.ph/metro-manila/pasig/san-miguel-80/house/buy/?page=12
https://www.lamudi.com.ph/metro-manila/pasig/san-miguel-80/house/buy/?page=13
https://www.lamudi.com.ph/metro-manila/pasig/san-miguel-80/house/buy/?page=14
https://www.lamudi.com.ph/metro-manila/pasig/san-miguel-80/house/buy/?page=15
https://www.lamudi.com.ph/metro-manila/pasig/san-miguel-80/house/buy/?page=16
https://www.lamudi.com.ph/metro-manila/pasig/san-miguel-80/house/buy/?page=17
https://www.lamudi.com.ph/metro-manila/pasig/san-miguel-80/house/buy/?page=18
https://www.lamudi.com.ph/metro-manila/pasig/san-miguel-80/house/buy/?page=19
https://www.lamudi.com.ph/metro-manila/pasig/san-miguel-80/house/b

None
https://www.lamudi.com.ph/metro-manila/makati/urdaneta-2/house/buy/?page=2
https://www.lamudi.com.ph/metro-manila/makati/urdaneta-2/house/buy/?page=3
https://www.lamudi.com.ph/metro-manila/makati/urdaneta-2/house/buy/?page=4
https://www.lamudi.com.ph/metro-manila/makati/urdaneta-2/house/buy/?page=5
None
https://www.lamudi.com.ph/metro-manila/pasay/house/buy/?page=2
None
https://www.lamudi.com.ph/metro-manila/taguig/house/buy/?page=2
https://www.lamudi.com.ph/metro-manila/taguig/house/buy/?page=3
https://www.lamudi.com.ph/metro-manila/taguig/house/buy/?page=4
https://www.lamudi.com.ph/metro-manila/taguig/house/buy/?page=5
https://www.lamudi.com.ph/metro-manila/taguig/house/buy/?page=6
https://www.lamudi.com.ph/metro-manila/taguig/house/buy/?page=7
https://www.lamudi.com.ph/metro-manila/taguig/house/buy/?page=8
https://www.lamudi.com.ph/metro-manila/taguig/house/buy/?page=9
https://www.lamudi.com.ph/metro-manila/taguig/house/buy/?page=10
https://www.lamudi.com.ph/metro-manila/taguig

https://www.lamudi.com.ph/metro-manila/paranaque/don-bosco/house/buy/?page=36
https://www.lamudi.com.ph/metro-manila/paranaque/don-bosco/house/buy/?page=37
https://www.lamudi.com.ph/metro-manila/paranaque/don-bosco/house/buy/?page=38
https://www.lamudi.com.ph/metro-manila/paranaque/don-bosco/house/buy/?page=39
https://www.lamudi.com.ph/metro-manila/paranaque/don-bosco/house/buy/?page=40
https://www.lamudi.com.ph/metro-manila/paranaque/don-bosco/house/buy/?page=41
https://www.lamudi.com.ph/metro-manila/paranaque/don-bosco/house/buy/?page=42
https://www.lamudi.com.ph/metro-manila/paranaque/don-bosco/house/buy/?page=43
https://www.lamudi.com.ph/metro-manila/paranaque/don-bosco/house/buy/?page=44
https://www.lamudi.com.ph/metro-manila/paranaque/don-bosco/house/buy/?page=45
None
https://www.lamudi.com.ph/metro-manila/paranaque/moonwalk/house/buy/?page=2
https://www.lamudi.com.ph/metro-manila/paranaque/moonwalk/house/buy/?page=3
https://www.lamudi.com.ph/metro-manila/paranaque/moonwalk/house

https://www.lamudi.com.ph/metro-manila/las-pinas/almanza-dos/house/buy/?page=16
https://www.lamudi.com.ph/metro-manila/las-pinas/almanza-dos/house/buy/?page=17
https://www.lamudi.com.ph/metro-manila/las-pinas/almanza-dos/house/buy/?page=18
https://www.lamudi.com.ph/metro-manila/las-pinas/almanza-dos/house/buy/?page=19
https://www.lamudi.com.ph/metro-manila/las-pinas/almanza-dos/house/buy/?page=20
https://www.lamudi.com.ph/metro-manila/las-pinas/almanza-dos/house/buy/?page=21
https://www.lamudi.com.ph/metro-manila/las-pinas/almanza-dos/house/buy/?page=22
https://www.lamudi.com.ph/metro-manila/las-pinas/almanza-dos/house/buy/?page=23
https://www.lamudi.com.ph/metro-manila/las-pinas/almanza-dos/house/buy/?page=24
None
https://www.lamudi.com.ph/metro-manila/las-pinas/pilar-12/house/buy/?page=2
https://www.lamudi.com.ph/metro-manila/las-pinas/pilar-12/house/buy/?page=3
https://www.lamudi.com.ph/metro-manila/las-pinas/pilar-12/house/buy/?page=4
https://www.lamudi.com.ph/metro-manila/las-pina

https://www.lamudi.com.ph/metro-manila/muntinlupa/poblacion-362/house/buy/?page=6
https://www.lamudi.com.ph/metro-manila/muntinlupa/poblacion-362/house/buy/?page=7
https://www.lamudi.com.ph/metro-manila/muntinlupa/poblacion-362/house/buy/?page=8
https://www.lamudi.com.ph/metro-manila/muntinlupa/poblacion-362/house/buy/?page=9
None
None
https://www.lamudi.com.ph/metro-manila/muntinlupa/tunasan/house/buy/?page=2
https://www.lamudi.com.ph/metro-manila/muntinlupa/tunasan/house/buy/?page=3
https://www.lamudi.com.ph/metro-manila/muntinlupa/tunasan/house/buy/?page=4
None
https://www.lamudi.com.ph/metro-manila/muntinlupa/cupang-4/house/buy/?page=2
https://www.lamudi.com.ph/metro-manila/muntinlupa/cupang-4/house/buy/?page=3
https://www.lamudi.com.ph/metro-manila/muntinlupa/cupang-4/house/buy/?page=4
https://www.lamudi.com.ph/metro-manila/muntinlupa/cupang-4/house/buy/?page=5
https://www.lamudi.com.ph/metro-manila/muntinlupa/cupang-4/house/buy/?page=6
https://www.lamudi.com.ph/metro-manila/munti

In [13]:
# dataframe creation
data = pd.DataFrame({
    "City" : cities,
    "Title" : titles,
    "Address" : addresses,
    "Listing Link" : listing_links,
    "Price" : prices,
    "Latitude" : latitudes,
    "Longitude" : longitudes, 
    "Floor Area" : floor_areas,
    "Land Area" : land_areas,
    "Number of Bathrooms" : number_bathrooms,
    "Number of Bedrooms" : number_bedrooms,
    "Subcategories" : subcategories,
    "Agent Name" : agent_names,
    "Verification" : verifications
})

In [14]:
data

,City,Title,Address,Listing Link,Price,Latitude,Longitude,Floor Area,Land Area,Number of Bathrooms,Number of Bedrooms,Subcategories,Agent Name,Verification
0,metro-manila/caloocan,3 Bedrooms House with Garage For Sale in Lilie...,"Camarin, Caloocan",https://www.lamudi.com.ph/3-bedrooms-house-wit...,5600000.0,14.759571,121.463220,97,50,3,3,"[""house"",""single-family-house""]",MARY LE GRACE RIVERA,Fully Verified
1,metro-manila/caloocan,3 Bedroom House & Lot in Amparo Caloocan sells...,"Amparo, Caloocan",https://www.lamudi.com.ph/3-bedroom-house-lot-...,5340000.0,14.747185,121.076954,83,53,2,3,"[""house"",""single-family-house""]",Genifel Pagulayan,Fully Verified
2,metro-manila/caloocan,10% Downpayment House For Sale In Amparo Subdi...,"Amparo, Caloocan",https://www.lamudi.com.ph/10-downpayment-house...,4694800.0,14.747185,121.076954,79,74,2,3,"[""house"",""townhouse""]",Evelyn Samaniego,Fully Verified
3,metro-manila/caloocan,"Unit 2, Manga Residences Townhouse For Sale in...","Amparo, Caloocan",https://www.lamudi.com.ph/unit-2-manga-residen...,5950000.0,14.747185,121.076954,123.52,76.91,3,3,"[""house"",""townhouse""]",Evelyn Samaniego,Fully Verified
4,metro-manila/caloocan,"10% Downpayment Townhouse for sale in Amparo,...","Amparo, Caloocan",https://www.lamudi.com.ph/10-downpayment-townh...,6525000.0,14.747185,121.076954,103,55,2,3,"[""house"",""townhouse""]",Evelyn Samaniego,Fully Verified
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24169,metro-manila/pateros,For Sale 2-Storey 3 Bedroom Single-family Hous...,"Comembo, Makati",https://www.lamudi.com.ph/for-sale-2-storey-3-...,18000000.0,14.546721,121.064315,157.38,122.8,2,3,"[""house"",""single-family-house""]",Percival Asher Ramirez,Basic
24170,metro-manila/pateros,3 Bedroom House and Lot For Sale in Makati Cit...,"East Rembo, Makati",https://www.lamudi.com.ph/3-bedroom-house-and-...,28000000.0,14.552973,121.061458,165,244,3,4,"[""house"",""single-family-house""]",Kai Lazaro Marquez,Basic
24171,metro-manila/pateros,4 Bedrooms House and Lot for sale at East Remb...,"East Rembo, Makati",https://www.lamudi.com.ph/4-bedrooms-house-and...,12000000.0,14.552973,121.061458,220,75,3,4,"[""house"",""single-family-house""]",Tristan Veracion,Basic
24172,metro-manila/pateros,House and Lot or Compound with Townhouse for S...,"East Rembo, Makati",https://www.lamudi.com.ph/house-and-lot-or-com...,44320000.0,14.552973,121.061458,300,277,3,7,"[""house"",""single-family-house""]",Christmas Morena,Fully Verified


In [15]:
data["Subcategories"].unique()

array(['["house","single-family-house"]', '["house","townhouse"]',
       '["house","villas"]', '["house","beach-house"]'], dtype=object)

In [16]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24174 entries, 0 to 24173
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City                 24174 non-null  object 
 1   Title                24174 non-null  object 
 2   Address              24174 non-null  object 
 3   Listing Link         24174 non-null  object 
 4   Price                24174 non-null  float64
 5   Latitude             24174 non-null  float64
 6   Longitude            24174 non-null  float64
 7   Floor Area           23884 non-null  object 
 8   Land Area            23988 non-null  object 
 9   Number of Bathrooms  23690 non-null  object 
 10  Number of Bedrooms   24014 non-null  object 
 11  Subcategories        24174 non-null  object 
 12  Agent Name           24174 non-null  object 
 13  Verification         24174 non-null  object 
dtypes: float64(3), object(11)
memory usage: 2.6+ MB


In [17]:
data["Verification"].value_counts()

Verification
Fully Verified    21751
Semi Verified      1848
Basic               575
Name: count, dtype: int64

In [18]:
data["City"].value_counts()

City
metro-manila/paranaque/b-f-homes           1500
metro-manila/quezon-city/fairview          1500
metro-manila/paranaque/don-bosco           1350
metro-manila/quezon-city/tandang-sora-3    1339
metro-manila/pasig/san-miguel-80           1215
                                           ... 
metro-manila/makati/legazpi-village-1         6
metro-manila/muntinlupa/buli-3                6
metro-manila/makati/salcedo-village-1         6
metro-manila/navotas                          6
metro-manila/pateros                          5
Name: count, Length: 67, dtype: int64

In [21]:
result_qc = data[data['City'].str.startswith('metro-manila/quezon-city/')]
result_qc

,City,Title,Address,Listing Link,Price,Latitude,Longitude,Floor Area,Land Area,Number of Bathrooms,Number of Bedrooms,Subcategories,Agent Name,Verification
1136,metro-manila/quezon-city/fairview,"East Fairview House and Lot 3 Bedrooms, 3 Toil...","Fairview, Quezon City",https://www.lamudi.com.ph/east-fairview-house-...,8350000.0,14.710380,121.072570,102,68,3,3,"[""house"",""single-family-house""]",Louella Laranang,Fully Verified
1137,metro-manila/quezon-city/fairview,RFO 2 Storey Townhouse for sale in East Fairvi...,"Fairview, Quezon City",https://www.lamudi.com.ph/rfo-2-storey-townhou...,6200000.0,14.702178,121.068210,100,30,3,3,"[""house"",""townhouse""]",Great Empire Realty,Fully Verified
1138,metro-manila/quezon-city/fairview,Affordable Townhouse for sale in West Fairview...,"Fairview, Quezon City",https://www.lamudi.com.ph/affordable-4-bedroom...,5150000.0,14.702178,121.068210,77,44.6,2,3,"[""house"",""townhouse""]",Great Empire Realty,Fully Verified
1139,metro-manila/quezon-city/fairview,3-Storey Townhouse For Sale in North Olympus S...,"Fairview, Quezon City",https://www.lamudi.com.ph/3-storey-townhouse-f...,5710000.0,14.702178,121.068210,116,43,3,3,"[""house"",""townhouse""]",Genifel Pagulayan,Fully Verified
1140,metro-manila/quezon-city/fairview,Single Attached House 3 Storey for Sale in Wes...,"Fairview, Quezon City",https://www.lamudi.com.ph/single-attached-hous...,9850000.0,14.702178,121.068210,170.42,74.5,5,6,"[""house"",""single-family-house""]",Jojo Obra,Fully Verified
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7911,metro-manila/quezon-city/teachers-village-east,Quezon City Townhouse for Sale Last Unit Left ...,"Teachers Village East, Quezon City",https://www.lamudi.com.ph/quezon-city-townhous...,14800000.0,14.643699,121.060025,166,60,4,3,"[""house"",""townhouse""]",Miss Sandy,Fully Verified
7912,metro-manila/quezon-city/teachers-village-east,Teachers Village Quezon City Townhouse with 2 ...,"Teachers Village East, Quezon City",https://www.lamudi.com.ph/teachers-village-que...,14600000.0,14.643699,121.060025,166,60,3,3,"[""house"",""townhouse""]",Miyabi Realty,Fully Verified
7913,metro-manila/quezon-city/teachers-village-east,22M House & Lot For Sale in Teachers Village Q...,"Teachers Village East, Quezon City",https://www.lamudi.com.ph/22m-house-lot-for-sa...,22000000.0,14.643699,121.060025,140,150,3,3,"[""house"",""single-family-house""]",Ms. Shine,Fully Verified
7914,metro-manila/quezon-city/teachers-village-east,3 Storey Townhouse with Big Roofdeck For Sale ...,"Teachers Village East, Quezon City",https://www.lamudi.com.ph/3-storey-townhouse-w...,23600000.0,14.643699,121.060025,277.06,68,4,3,"[""house"",""townhouse""]",Miss Sandy,Fully Verified


In [22]:
result_qc["City"].value_counts()

City
metro-manila/quezon-city/fairview                 1500
metro-manila/quezon-city/tandang-sora-3           1339
metro-manila/quezon-city/project-8                 768
metro-manila/quezon-city/batasan-hills             678
metro-manila/quezon-city/cubao-1                   568
metro-manila/quezon-city/diliman                   537
metro-manila/quezon-city/holy-spirit               535
metro-manila/quezon-city/commonwealth              510
metro-manila/quezon-city/teachers-village-east     345
Name: count, dtype: int64

In [19]:
data.to_csv("Housing Price data.csv")

In [ ]:
# Create a geometry column
geometry = [Point(xy) for xy in zip(data['Longitude'], data['Latitude'])]

# Create a GeoDataFrame
gdf = gpd.GeoDataFrame(data, geometry=geometry)

# Optional: Set the coordinate reference system (CRS) to EPSG:4326 (WGS84) for latitude and longitude data
gdf.crs = "EPSG:4326"

gdf.to_file("Housing Price data.shp")

In [ ]:
import geopandas as gpd
gdf = gpd.GeoDataFrame(data, geometry=gpd.points_from_xy(data["Longitude"], data["Latitude"]))
gdf